# TEE - 数据安全新范式

Laisky Cai

In [ ]:
本次分享主要想介绍一下基于硬件的机密计算 TEE 领域，这很可能代表着下一个时代的标准安全基座。

本次主要是概念普及和基础原理介绍，不需要相关背景知识。

* 本 slide 的在线浏览地址：<https://s3.laisky.com/public/slides/TEE-101.slides.html>
    
* 相关术语表: <https://wiki.laisky.com/terms/>

![](https://s3.laisky.com/uploads/2023/04/tee-101/qrcode.png)

## 数据

机密计算（CC, Confidential Computing）是一种能够依托硬件安全能力，保护用户数据的技术统称。

在讲解机密计算以前，先了解一下数据。

## 运行环境的变迁

### 场景 1：数据加密

### 场景 2：密钥生成

### 场景 3：隐私数据出域

## 可信计算

假设你有一台电脑，无论是个人 PC 还是服务器，你在上面运行一个操作系统，然后在操作系统上运行一个软件。

你该如何确保你的系统和软件没有被入侵？

注：我们不讨论和黑客实时攻防等很酷炫的招数。而是从更普遍的运维和开发的角度出发，你肯定没精力去和黑客搞对抗的。

你更希望的是：正确安装和运行一个没有已知漏洞的系统，运行一个精心编写的应用程序。

而可信计算所关注的问题正是：向你保证你所运行的操作系统是可以信赖的。

## TPM

为了确保系统启动过程的全流程可信，就需要这个小小的 TPM（Trusted Platform Module） 硬件模块了。

![](https://s3.laisky.com/uploads/2023/04/tee-101/tpm2.0.jpg)

### 系统如何启动

要想确保运行的系统是可信的，需要先了解一下系统的启动过程

1. 系统加电、加载 BIOS Boot Block
2. BIOS/UEFI 开始运行，激活各个协处理器、硬件
3. BIOS 执行 kernel loader，加载 OS
4. OS 启动后运行 init 进程，该进程按照配置启动各个 App 进程

### Measurement

![](https://s3.laisky.com/uploads/2023/04/tee-101/pcr.png)

TPM 中包含了数个只读寄存器，称为 PCRs（Platform Configuration Registers）。

每当系统重启时，所有的 PCR 都会被清空。然后在系统启动过程的不同阶段，
各个 PCR 按照预设的程序会对系统当前的状态进行度量（Measurement），
并且将度量值写入到对应的 PCR 之中。

可以用一段伪代码来描述 tpm 的行为

```go
// TPM PCR 的行为几乎和 hash 函数一模一样
hasher := sha256.New()

<- bootblock.Ready()
hasher.Write(bootblock.Measure())
tpm.SetPCR(1, hasher.Sum(nil))

<- bios.Ready()
hasher.Write(bios.Measure())
tpm.SetPCR(2, hasher.Sum(nil))

<- os.Ready()
hasher.Write(os.Measure())
tpm.SetPCR(3, hasher.Sum(nil))

...
```

`PCR[N] = HASHalg( PCR[N] || ArgumentOfExtend)`

[TCG PC Client Platform TPM Profile Specification for TPM 2.0.pdf] 规范要求 TPM 2.0 硬件应该至少支持 24 个 PCRs，
且支持 SHA256 和 SHA384 算法。

![](https://s3.laisky.com/uploads/2023/04/tee-101/pcrs.png)

前面提到 PCRs 是只读寄存器，所以在系统启动过程中生成的 PCR 值是不可篡改的。
在制作系统之时，我们可以将 PCR 的值记录下来，然后在日后系统启动时进行校验，所有的 PCRs 都不应发生任何变化。

实际上我们并不需要手动去记录和校验 PCR 的值。TPM 除了度量外还内置了密钥生成的功能，
它可以根据利用设备的硬件私钥和当前 PCR 值派生一个用户密钥。
只要是同一台 TPM 设备，只要系统的 PCR 值没有发生变化，那么派生出来的密钥也是一样的。

Windows 磁盘加密工具 BitLocker 就利用了 TPM 的这一功能。
它会在系统启动时，根据当前的 PCR 值派生一个密钥，然后用这个密钥对磁盘进行加密。
一旦系统被篡改，就会导致 PCR 发生变化，那么就无法再次派生出相同的密钥，从而无法解密磁盘。

### 系统安全的范畴

![](https://s3.laisky.com/uploads/2023/04/tee-101/tcb-base.jpeg)

## 云时代的安全新需求

过去人们在自己的机器上处理数据，机器和局域网完全在自己的掌控之中，
理论上只要做好外围安全防护，就不用担心数据安全问题。

但是随着进入云时代，数据量和计算量越来越大，人们需要利用云上的可扩展性计算资源来处理海量数据。
这就导致人们需要将自己的程序和数据传输到云上，涉及到了传输安全、存储安全和计算安全的问题。

在系统安全的范畴下，侧重考虑安全的两层含义：

* 完整性 Integrity
* 保密性 Confidential

**完整性**用于监测数据不被篡改。如代码完整性意味着在云上运行的应用确实是自己构建的应用，
应用没有被任何程度的篡改，程序运行行为符合预期。

操作系统完整性则表明操作系统没有被篡改，程序运行于正确、安全的操作系统之中。

**机密性**意味着数据明文不为外人所窥视，仅有持有密钥的参与者可以解密读取明文。

在云场景下，完整性确保云上运行的确实是自己的应用，输入的确实是自己的数据。
而保密性则确保程序所处理的数据以及程序本身（算子）不被外人窥探。

### 云的安全现状

现代的云服务提供商（CSP）一般利用 虚拟机（VM）/runC 等技术提供多租户支持，
租户的运行环境被限制于沙盒（Sandbox、SFI）之中，无法访问沙盒外的宿主机资源，租户与租户之间也相互隔离。

但是就算我们认为这套沙盒隔离机制是有效的，实际上也丝毫没有提高租户的安全性，
更多的只是提高了 CSP 平台自身的安全性。

![](https://s3.laisky.com/uploads/2023/04/tee-101/csp-current.drawio.png)

沙盒内的租户难以从沙盒中逃逸，但是沙盒本身对于宿主机 OS 而言就是一个普通的进程，
其运行时数据完全是明文的，宿主机 OS 也可以随意访问沙盒内的数据。

简而言之，**云上应用可谓毫无安全性可言**。

Ps. 至于有些 CSP 声称提供了所谓的可信计算环境，在后续文章中会进行批判😂。